In [1]:
# Cell 1 — Setup (install deps)
# If you're on Colab, uncomment the next lines.
# !pip -q install sbmltoodepy python-libsbml numpy scipy matplotlib

import os, io, zipfile, pathlib, textwrap, sys, json
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request

print("Ready.")


Ready.


In [3]:
# Cell 2 — Robust download of Chen 2004 SBML (BIOMD0000000056)
# Tries current BioModels endpoints; falls back to manual if needed.

import os, pathlib, sys, time
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError

TARGET = "chen2004_biomd56.xml"

# Known-good endpoints as of 2025-09-20:
CANDIDATES = [
    # Direct "services/download" link for this model's internal ID + version 3
    "https://www.ebi.ac.uk/biomodels/services/download/get-files/MODEL6624073334/3/BIOMD0000000056_url.xml",
    # Generic download endpoint without the (now-retired) .4 suffix
    "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000056?filename=BIOMD0000000056_url.xml",
]

UA = {"User-Agent": "Mozilla/5.0"}

def try_download(url):
    try:
        with urlopen(Request(url, headers=UA), timeout=30) as r:
            data = r.read()
        with open(TARGET, "wb") as f:
            f.write(data)
        print(f"Downloaded SBML from:\n  {url}\n→ saved as {TARGET} ({len(data)} bytes)")
        return True
    except HTTPError as e:
        print(f"[HTTP {e.code}] {url}")
    except URLError as e:
        print(f"[URL error] {url} :: {e.reason}")
    except Exception as e:
        print(f"[Other error] {url} :: {e}")
    return False

if not os.path.exists(TARGET):
    ok = False
    for url in CANDIDATES:
        if try_download(url):
            ok = True
            break
        time.sleep(0.8)  # polite pause
    if not ok:
        raise SystemExit(
            "Could not fetch SBML automatically.\n"
            "Manual fallback: open the model page in a browser, "
            "click Files → 'BIOMD0000000056_url.xml' → Download, "
            "and save it as 'chen2004_biomd56.xml' in this notebook folder."
        )
else:
    print(f"Found existing {TARGET}, reusing it.")



Downloaded SBML from:
  https://www.ebi.ac.uk/biomodels/services/download/get-files/MODEL6624073334/3/BIOMD0000000056_url.xml
→ saved as chen2004_biomd56.xml (208287 bytes)


In [7]:
# Cell 3 — Convert SBML → pure-Python ODEs using sbmltoodepy
# This generates a module with:
#   - a class 'Model' exposing state names, parameter dict, initial conditions
#   - an ODE RHS function compatible with SciPy

from sbmltoodepy import sbml_to_ode

out_dir = pathlib.Path("chen_model_py")
out_dir.mkdir(exist_ok=True)

sbml_to_ode.convert("chen2004_biomd56.xml", output_dir=str(out_dir), python_module_name="chen2004_model")

print("Generated files:", list(out_dir.iterdir()))


ModuleNotFoundError: No module named 'sbmltoodepy'

In [8]:
!pip install sbmltoodepy python-libsbml

/opt/anaconda3/bin/python


In [ ]:
# Cell 4 — Inspect the generated model: states, parameters, initial conditions

sys.path.insert(0, str(out_dir))
import chen2004_model as cm

m = cm.Model()  # generated class
state_names = m.get_state_names()
y0 = m.get_initial_conditions_vector()
params = m.get_parameter_dict()

print("Number of ODEs:", len(state_names))
print("First 15 states:", state_names[:15])
print("Total parameters:", len(params))
print("Example parameters:", list(params.items())[:10])


In [9]:
# In a notebook cell (base is fine)
!pip install tellurium

import tellurium as te
rr = te.loadSBMLModel("chen2004_biomd56.xml")   # the file you downloaded
result = rr.simulate(0, 200, 2001)
rr.plot(result)


     |████████████████████████████████| 128 kB 3.2 MB/s eta 0:00:01
  Using cached python_libsbml-5.20.5.tar.gz (41.8 MB)
     |████████████████████████████████| 3.1 MB 33.0 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 61 kB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 56.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 46.0 MB 715 kB/s eta 0:00:011     |██████████████▍                 | 20.6 MB 3.1 MB/s eta 0:00:09
     |████████████████████████████████| 1.9 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 70.1 MB 115 kB/s eta 0:00:01     |████████████████████████▌       | 53.6 MB 211 kB/s eta 0:01:19
  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)


  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/dg/j3xfsm6j68s_0jk_l6mqmh_h0000gn/T/pip-install-9z8en92h/python-libnuml_035b8f9641514c998a459e83cb22c1e6/setup.py'"'"'; __file__='"'"'/private/var/folders/dg/j3xfsm6j68s_0jk_l6mqmh_h0000gn/T/pip-install-9z8en92h/python-libnuml_035b8f9641514c998a459e83cb22c1e6/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/dg/j3xfsm6j68s_0jk_l6mqmh_h0000gn/T/pip-wheel-5hdhqu8t
       cwd: /private/var/folders/dg/j3xfsm6j68s_0jk_l6mqmh_h0000gn/T/pip-install-9z8en92h/python-libnuml_035b8f9641514c998a459e83cb22c1e6/
  Complete output (24 lines):
  Using libNuML from: /private/

Failed to build python-libnuml python-libsbml python-libsedml
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
    Running setup.py install for python-libsedml ... error
    ERROR: Command errored out with exit status 1:
     command: /opt/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/dg/j3xfsm6j68s_0jk_l6mqmh_h0000gn/T/pip-install-9z8en92h/python-libsedml_aaab0abfffd24219bdc012bcef85aa49/setup.py'"'"'; __file__='"'"'/private/var/folders/dg/j3xfsm6j68s_0jk_l6mqmh_h0000gn/T/pip-install-9z8en92h/python-libsedml_aaab0abfffd24219bdc012bcef85aa49/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --rec

ModuleNotFoundError: No module named 'tellurium'